In [1]:
!pip install sqlalchemy psycopg2 pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 31.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 55.9 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  DEPRECATION: psycopg2 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for psycopg2 ... done

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sqlalchemy import create_engine, MetaData
import pandas as pd
import json

# SQLite source
sqlite_engine = create_engine('sqlite:///./database.db')
sqlite_metadata = MetaData()
sqlite_metadata.reflect(bind=sqlite_engine)

user = "padel_user"
password = "portopadelleague"
host = "35.205.246.86"  # your GCE VM public IP
port = "5432"
database = "padel_league"

# PostgreSQL destination
pg_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

# Migrate each table
for table_name, table in sqlite_metadata.tables.items():
    print(f"Migrating: {table_name}")
    df = pd.read_sql_table(table_name, sqlite_engine)

    # Convert any dict columns to JSON strings
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, dict)).any():
            print(f"  ➤ Converting column '{col}' to JSON")
            df[col] = df[col].apply(lambda d: json.dumps(d) if isinstance(d, dict) else d)

    df.to_sql(table_name, pg_engine, if_exists='replace', index=False)

print("✅ Migration complete!")


Migrating: divisions
Migrating: editions
Migrating: leagues
Migrating: matches
Migrating: news
Migrating: order_lines
Migrating: orders
Migrating: users
Migrating: players
Migrating: products
  ➤ Converting column 'features' to JSON
Migrating: players_in_division
Migrating: players_in_match
Migrating: product_attribute_values
Migrating: product_attributes
Migrating: product_attribute_values_in_product
Migrating: product_attributes_in_product
Migrating: product_images
Migrating: registrations
✅ Migration complete!


In [5]:
from sqlalchemy import inspect

inspector = inspect(pg_engine)
tables = inspector.get_table_names()
print(tables)

['divisions', 'editions', 'leagues', 'matches', 'news', 'order_lines', 'orders', 'users', 'players', 'products', 'players_in_division', 'players_in_match', 'product_attribute_values', 'product_attributes', 'product_attribute_values_in_product', 'product_attributes_in_product', 'product_images', 'registrations']


In [ ]:
df = pd.read_sql("SELECT * FROM products", pg_engine)
df.head()